In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
lr = 1e-4
env = gym.make('CartPole-v0')
env.reset()

[2018-01-22 17:28:14,371] Making new env: CartPole-v0


array([ 0.04125665,  0.02371672,  0.03806007,  0.03194964])

In [4]:
goal_steps = 500
score_requirement = 50
initial_games = 25000

In [ ]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            # takes a random action from the action space
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break

In [ ]:
some_random_games_first()

In [5]:
# generate training samples
def initial_population():
    # stores the observations that a random move made
    training_data = []
    scores = []
    # appends the data only if the corresponding score happens to be above 50
    accepted_scores = []
    for i in range(initial_games):
        score = 0
        # store game memory as we will not know until the end of the game whether or not we beat the required score
        game_memory = []
        prev_observation = []
        # iterate through the plausible steps
        for j in range(goal_steps):
            action = random.randrange(0, 2) # generates a 0 or a 1
            observation, reward, done, info = env.step(action)
            
            if len(prev_observation) > 0:
                # basically looks at the previous frame and stores the current action and if the score is high enough,
                # it will be appended to the training data as we looked at the previous frame and did something on
                # this frame that increased our score, and we would like our neural network to learn this mapping
                game_memory.append([prev_observation, action])
                
            prev_observation = observation
            score += reward
            if done:
                break
        
        if score >= score_requirement:
            accepted_scores.append(score)
            # converting output into a one-hot vector for training
            for data in game_memory:
                if data[1] == 1:
                    output = [0, 1]
                elif data[1] == 0:
                    output = [1, 0]
                    
                # appends the observation and the given output(in one-hot vector format)
                training_data.append([data[0], output])
                
        env.reset()
        scores.append(score)
    
    training_data_save = np.array(training_data)
    np.save('cartpole-v0_training_data.npy', training_data_save)
    
    print('Average accepted score: ', mean(accepted_scores))
    print('Median accepted score: ', median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [11]:
initial_population()

[2018-01-22 12:41:14,917] You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


Average accepted score:  60.79389312977099
Median accepted score:  57.0
Counter({51.0: 37, 54.0: 33, 50.0: 33, 52.0: 25, 55.0: 21, 56.0: 19, 60.0: 18, 57.0: 17, 53.0: 17, 62.0: 17, 59.0: 12, 67.0: 11, 58.0: 11, 65.0: 11, 61.0: 11, 70.0: 9, 63.0: 9, 71.0: 8, 64.0: 7, 69.0: 6, 77.0: 6, 78.0: 6, 74.0: 5, 68.0: 5, 76.0: 4, 66.0: 3, 93.0: 3, 75.0: 3, 90.0: 3, 73.0: 2, 89.0: 2, 83.0: 2, 85.0: 2, 81.0: 2, 84.0: 1, 118.0: 1, 109.0: 1, 80.0: 1, 79.0: 1, 91.0: 1, 94.0: 1, 110.0: 1, 92.0: 1, 88.0: 1, 87.0: 1, 86.0: 1, 98.0: 1})


[[array([-0.00597893, -0.20881766, -0.03947159,  0.2866333 ]), [0, 1]],
 [array([-0.01015528, -0.01315568, -0.03373893, -0.01823271]), [0, 1]],
 [array([-0.0104184 ,  0.18243348, -0.03410358, -0.32136684]), [1, 0]],
 [array([-0.00676973, -0.01218664, -0.04053092, -0.03963092]), [0, 1]],
 [array([-0.00701346,  0.18349238, -0.04132354, -0.34482122]), [0, 1]],
 [array([-0.00334361,  0.37917707, -0.04821996, -0.65024333]), [0, 1]],
 [array([ 0.00423993,  0.57493633, -0.06122483, -0.95771209]), [1, 0]],
 [array([ 0.01573865,  0.3806887 , -0.08037907, -0.68487543]), [1, 0]],
 [array([ 0.02335243,  0.18676928, -0.09407658, -0.41854145]), [0, 1]],
 [array([ 0.02708781,  0.38308966, -0.10244741, -0.73933759]), [1, 0]],
 [array([ 0.03474961,  0.18952029, -0.11723416, -0.48057278]), [1, 0]],
 [array([ 0.03854001, -0.00376857, -0.12684561, -0.22701755]), [1, 0]],
 [array([ 0.03846464, -0.19687106, -0.13138596,  0.02311639]), [1, 0]],
 [array([ 0.03452722, -0.38988803, -0.13092364,  0.27163005]), [

In [6]:
def neural_network_model(input_size):
    network = input_data(shape=[None, input_size, 1], name='input')
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8) # keep_rate=0.8
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')
    return model

In [12]:
def train_model(training_data, model=False):
    X = np.array([i[0] for i in training_data]).reshape(-1, len(training_data[0][0]), 1) # grabs the observation column of the training data
    y = [i[1] for i in training_data]
    
    if not model:
        model = neural_network_model(input_size=len(X[0]))
    
    model.fit({'input':X}, {'targets':y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id='cartpole-v0-v0.00')
    return model

In [13]:
training_data = initial_population()
model = train_model(training_data)

[2018-01-22 17:38:47,611] You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


Average accepted score:  61.293577981651374
Median accepted score:  58.0
Counter({50.0: 77, 53.0: 75, 51.0: 63, 52.0: 55, 55.0: 50, 54.0: 50, 57.0: 33, 56.0: 31, 60.0: 31, 58.0: 29, 62.0: 28, 59.0: 27, 67.0: 27, 61.0: 26, 63.0: 25, 65.0: 20, 64.0: 19, 66.0: 18, 68.0: 17, 72.0: 14, 70.0: 11, 69.0: 11, 74.0: 10, 73.0: 9, 82.0: 9, 76.0: 9, 77.0: 8, 75.0: 8, 71.0: 7, 79.0: 6, 83.0: 6, 78.0: 6, 81.0: 5, 86.0: 5, 84.0: 5, 88.0: 5, 80.0: 4, 100.0: 3, 108.0: 2, 89.0: 2, 96.0: 2, 105.0: 2, 98.0: 2, 95.0: 2, 90.0: 2, 91.0: 2, 85.0: 2, 92.0: 1, 94.0: 1, 118.0: 1, 104.0: 1, 87.0: 1, 102.0: 1, 101.0: 1, 99.0: 1, 106.0: 1, 110.0: 1, 124.0: 1, 107.0: 1})
---------------------------------
Run id: cartpole-v0-v0.00
Log directory: log/
INFO:tensorflow:Summary name Accuracy/Adam_0 (raw) is illegal; using Accuracy/Adam_0__raw_ instead.


[2018-01-22 17:39:21,531] Summary name Accuracy/Adam_0 (raw) is illegal; using Accuracy/Adam_0__raw_ instead.


INFO:tensorflow:Summary name Accuracy_1/Adam_1 (raw) is illegal; using Accuracy_1/Adam_1__raw_ instead.


[2018-01-22 17:39:21,562] Summary name Accuracy_1/Adam_1 (raw) is illegal; using Accuracy_1/Adam_1__raw_ instead.


---------------------------------
Training samples: 105152
Validation samples: 0
--


InvalidArgumentError: You must feed a value for placeholder tensor 'input_1/X' with dtype float and shape [?,4,1]
	 [[Node: input_1/X = Placeholder[dtype=DT_FLOAT, shape=[?,4,1], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'input_1/X', defined at:
  File "C:\Users\Aman Deep Singh\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Aman Deep Singh\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Aman Deep Singh\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Aman Deep Singh\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Aman Deep Singh\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Aman Deep Singh\Anaconda3\lib\site-packages\zmq\eventloop\iolo

In [9]:
model.save('cartpole-v0-v0.10')

INFO:tensorflow:C:\Users\Aman Deep Singh\Documents\Python\Data Science\Machine Learning\Reinforcement Learning\cartpole-v0-v0.10 is not in all_model_checkpoint_paths. Manually adding it.


[2018-01-22 17:32:20,862] C:\Users\Aman Deep Singh\Documents\Python\Data Science\Machine Learning\Reinforcement Learning\cartpole-v0-v0.10 is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
scores = []
choices = []
for each_game in range(100):
    score = 0
    game_memory = []
    prev_observation = []
    env.reset()
    for _ in range(goal_steps):
        # env.render()
        if len(prev_observation) == 0:
            action = random.randrange(0, 2)
        else:
            # print(model.predict(prev_observation.reshape(-1, len(prev_observation), 1)))
            action = np.argmax(model.predict(prev_observation.reshape(-1, len(prev_observation), 1))[0])
            
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_observation = new_observation
        game_memory.append([new_observation, action])
        score += reward
        if done:
            break
    scores.append(score)
    
print('Average Score ', sum(scores)/len(scores))
print(f'Choice 1: {choices.count(1)/len(choices)}, Choice 0: {choices.count(0)/len(choices)}')

[[ 0.57397389  0.42602605]]
[[ 0.46915284  0.53084713]]
[[ 0.57493973  0.42506027]]
[[ 0.46986395  0.53013605]]
[[ 0.57662475  0.42337528]]
[[ 0.47124931  0.52875072]]
[[ 0.57901394  0.42098606]]
[[ 0.47333822  0.52666181]]
[[ 0.58216739  0.41783264]]
[[ 0.47611317  0.52388674]]
[[ 0.58607632  0.41392359]]
[[ 0.47961879  0.52038115]]
[[ 0.59081972  0.40918034]]
[[ 0.48393938  0.51606065]]
[[ 0.59647375  0.40352616]]
[[ 0.4891378   0.51086223]]
[[ 0.60316473  0.39683521]]
[[ 0.49543661  0.50456333]]
[[ 0.61101156  0.38898852]]
[[ 0.50322515  0.49677491]]
[[ 0.3988764   0.60112363]]
[[ 0.49604654  0.50395346]]
[[ 0.61045653  0.3895435 ]]
[[ 0.50325012  0.49674988]]
[[ 0.39869881  0.60130125]]
[[ 0.49566287  0.50433713]]
[[ 0.60897964  0.39102042]]
[[ 0.50229037  0.4977096 ]]
[[ 0.39759725  0.60240275]]
[[ 0.4944022   0.50559783]]
[[ 0.60663909  0.39336088]]
[[ 0.50039482  0.49960512]]
[[ 0.39567545  0.60432452]]
[[ 0.49222004  0.50777996]]
[[ 0.60334313  0.3966569 ]]
[[ 0.49762154  0.502

[[ 0.59062988  0.40937012]]
[[ 0.49193656  0.5080635 ]]
[[ 0.59163892  0.40836105]]
[[ 0.4938024   0.50619763]]
[[ 0.59308529  0.40691465]]
[[ 0.49606276  0.50393724]]
[[ 0.59496397  0.40503594]]
[[ 0.4987458   0.50125426]]
[[ 0.59726155  0.40273848]]
[[ 0.50187606  0.49812391]]
[[ 0.4074699   0.59253013]]
[[ 0.49169266  0.5083074 ]]
[[ 0.58827949  0.41172057]]
[[ 0.49288869  0.50711131]]
[[ 0.58885062  0.41114938]]
[[ 0.49433842  0.50566167]]
[[ 0.58974499  0.41025507]]
[[ 0.49595806  0.50404197]]
[[ 0.59095663  0.4090434 ]]
[[ 0.49784473  0.50215524]]
[[ 0.59250355  0.40749654]]
[[ 0.50006044  0.49993956]]
[[ 0.40762144  0.59237856]]
[[ 0.48976338  0.51023662]]
[[ 0.58190417  0.41809592]]
[[ 0.48990181  0.51009822]]
[[ 0.58129644  0.4187035 ]]
[[ 0.49013984  0.50986016]]
[[ 0.58080274  0.41919729]]
[[ 0.49046251  0.50953752]]
[[ 0.58037734  0.41962266]]
[[ 0.49085233  0.50914764]]
[[ 0.57997483  0.42002517]]
[[ 0.49130216  0.50869787]]
[[ 0.57962793  0.42037213]]
[[ 0.49179396  0.508

[[ 0.50290078  0.49709919]]
[[ 0.41617513  0.58382487]]
[[ 0.49577966  0.50422037]]
[[ 0.58364892  0.41635108]]
[[ 0.49907756  0.50092238]]
[[ 0.58480972  0.41519028]]
[[ 0.50292623  0.4970738 ]]
[[ 0.41851154  0.58148849]]
[[ 0.49549982  0.50450009]]
[[ 0.5778355   0.42216447]]
[[ 0.498106    0.50189394]]
[[ 0.57825094  0.42174911]]
[[ 0.50109524  0.49890476]]
[[ 0.4193078   0.58069217]]
[[ 0.4930847   0.50691527]]
[[ 0.5702967   0.42970333]]
[[ 0.49450034  0.50549966]]
[[ 0.56950659  0.43049341]]
[[ 0.49614906  0.503851  ]]
[[ 0.56874913  0.43125093]]
[[ 0.4980579  0.5019421]]
[[ 0.56806201  0.43193802]]
[[ 0.50024337  0.49975657]]
[[ 0.42403188  0.57596809]]
[[ 0.49207196  0.50792801]]
[[ 0.55947214  0.44052783]]
[[ 0.4925794   0.50742066]]
[[ 0.55718899  0.44281101]]
[[ 0.49312928  0.50687075]]
[[ 0.55472392  0.44527611]]
[[ 0.49371156  0.50628841]]
[[ 0.5519188   0.44808125]]
[[ 0.494178  0.505822]]
[[ 0.54894435  0.45105565]]
[[ 0.49463648  0.50536358]]
[[ 0.54577172  0.45422828]

[[ 0.49474442  0.50525558]]
[[ 0.53824103  0.46175891]]
[[ 0.48433617  0.51566386]]
[[ 0.53544933  0.4645507 ]]
[[ 0.47070462  0.52929538]]
[[ 0.5240919   0.47590807]]
[[ 0.45233771  0.5476622 ]]
[[ 0.50609857  0.49390137]]
[[ 0.43003643  0.56996351]]
[[ 0.48188588  0.51811409]]
[[ 0.54238772  0.45761222]]
[[ 0.46247768  0.53752232]]
[[ 0.52322221  0.47677785]]
[[ 0.43967426  0.56032568]]
[[ 0.49915507  0.50084496]]
[[ 0.56271684  0.43728316]]
[[ 0.48141322  0.5185867 ]]
[[ 0.54616493  0.45383507]]
[[ 0.46195567  0.53804439]]
[[ 0.52646756  0.47353241]]
[[ 0.4398154  0.5601846]]
[[ 0.50325394  0.49674603]]
[[ 0.41503114  0.58496886]]
[[ 0.47667852  0.52332151]]
[[ 0.54269248  0.45730758]]
[[ 0.45763555  0.54236442]]
[[ 0.52335513  0.47664487]]
[[ 0.43769717  0.56230283]]
[[ 0.50149566  0.49850431]]
[[ 0.41542113  0.58457893]]
[[ 0.47750819  0.52249181]]
[[ 0.54155606  0.45844394]]
[[ 0.46227345  0.53772658]]
[[ 0.5251801   0.47481996]]
[[ 0.44622445  0.55377549]]
[[ 0.50803798  0.49196

[[ 0.4677476   0.53225237]]
[[ 0.50407684  0.4959231 ]]
[[ 0.46990401  0.53009599]]
[[ 0.50429213  0.49570787]]
[[ 0.47237515  0.52762485]]
[[ 0.5046373  0.4953627]]
[[ 0.4751831   0.52481693]]
[[ 0.50520772  0.49479228]]
[[ 0.47824165  0.52175838]]
[[ 0.50541896  0.49458104]]
[[ 0.48160768  0.51839238]]
[[ 0.5051316   0.49486843]]
[[ 0.48477373  0.51522624]]
[[ 0.50440949  0.49559054]]
[[ 0.48766604  0.51233399]]
[[ 0.50327981  0.49672025]]
[[ 0.48979396  0.51020598]]
[[ 0.50195712  0.49804288]]
[[ 0.49135315  0.50864685]]
[[ 0.50094432  0.49905571]]
[[ 0.49222776  0.50777227]]
[[ 0.50009727  0.49990276]]
[[ 0.4925963   0.50740367]]
[[ 0.49944627  0.50055367]]
[[ 0.5072673   0.49273273]]
[[ 0.50155014  0.4984498 ]]
[[ 0.49368316  0.50631684]]
[[ 0.50114423  0.49885583]]
[[ 0.49398047  0.50601947]]
[[ 0.50099331  0.49900666]]
[[ 0.49445012  0.50554985]]
[[ 0.50079  0.49921]]
[[ 0.49511129  0.50488865]]
[[ 0.50051486  0.49948516]]
[[ 0.49591553  0.50408447]]
[[ 0.50024468  0.49975538]]


...

[[ 0.506006    0.49399403]]
[[ 0.4630219   0.53697807]]
[[ 0.50687689  0.49312305]]
[[ 0.46578777  0.53421217]]
[[ 0.50793409  0.49206588]]
[[ 0.46855876  0.53144121]]
[[ 0.50913543  0.49086455]]
[[ 0.47155818  0.52844185]]
[[ 0.51039243  0.48960754]]
[[ 0.47487584  0.52512413]]
[[ 0.51176846  0.48823154]]
[[ 0.47852808  0.52147192]]
[[ 0.51337957  0.4866204 ]]
[[ 0.48242068  0.51757926]]
[[ 0.51490539  0.48509458]]
[[ 0.48637545  0.51362455]]
[[ 0.51572871  0.48427126]]
[[ 0.49046609  0.50953394]]
[[ 0.51578236  0.4842177 ]]
[[ 0.49448663  0.50551343]]
[[ 0.51512003  0.48487991]]
[[ 0.49787182  0.50212818]]
[[ 0.51410592  0.48589399]]
[[ 0.50015867  0.49984133]]
[[ 0.48901924  0.51098078]]
[[ 0.49795547  0.50204456]]
[[ 0.50911409  0.49088585]]
[[ 0.49860126  0.50139874]]
[[ 0.5086022   0.49139777]]
[[ 0.49937603  0.500624  ]]
[[ 0.50830519  0.49169481]]
[[ 0.5002628   0.49973723]]
[[ 0.49292907  0.5070709 ]]
[[ 0.49880558  0.50119448]]
[[ 0.50554228  0.49445766]]
[[ 0.50006813  0.499

[[ 0.49349818  0.50650185]]
[[ 0.50012702  0.49987307]]
[[ 0.4934288   0.50657123]]
[[ 0.49964523  0.50035477]]
[[ 0.50338203  0.496618  ]]
[[ 0.49949232  0.50050765]]
[[ 0.50238889  0.49761117]]
[[ 0.49938002  0.50062001]]
[[ 0.50188106  0.49811894]]
[[ 0.49942672  0.50057328]]
[[ 0.50182718  0.49817291]]
[[ 0.49986395  0.50013614]]
[[ 0.50211132  0.49788868]]
[[ 0.50074691  0.49925312]]
[[ 0.49948761  0.50051236]]
[[ 0.50248623  0.49751377]]
[[ 0.50170422  0.49829584]]
[[ 0.5009554   0.49904466]]
[[ 0.49959573  0.50040418]]
[[ 0.50493509  0.49506488]]
[[ 0.50395423  0.4960458 ]]
[[ 0.50188839  0.49811161]]
[[ 0.49713802  0.50286192]]
[[ 0.50831997  0.49168009]]
[[ 0.50364244  0.4963575 ]]
[[ 0.49655056  0.5034495 ]]
[[ 0.51112258  0.48887739]]
[[ 0.50333112  0.49666893]]
[[ 0.49358812  0.50641185]]
[[ 0.51146889  0.4885312 ]]
[[ 0.49908844  0.50091153]]
[[ 0.52020222  0.47979778]]
[[ 0.5063467  0.4936533]]
[[ 0.48915291  0.51084703]]
[[ 0.51270539  0.48729461]]
[[ 0.48191217  0.51808

[[ 0.5026806  0.4973194]]
[[ 0.40143722  0.59856278]]
[[ 0.49375638  0.50624359]]
[[ 0.60693961  0.39306045]]
[[ 0.49789029  0.50210971]]
[[ 0.6118384   0.38816157]]
[[ 0.50300312  0.49699691]]
[[ 0.40220305  0.59779698]]
[[ 0.49421546  0.50578457]]
[[ 0.60687965  0.39312035]]
[[ 0.49831274  0.50168729]]
[[ 0.61172819  0.38827181]]
[[ 0.50341976  0.4965803 ]]
[[ 0.40301976  0.59698021]]
[[ 0.49465311  0.50534689]]
[[ 0.60676312  0.39323694]]
[[ 0.49870667  0.5012933 ]]
[[ 0.61149633  0.3885037 ]]
[[ 0.50372869  0.49627128]]
[[ 0.40368307  0.59631687]]
[[ 0.49493173  0.50506824]]
[[ 0.60640424  0.39359578]]
[[ 0.49887326  0.50112665]]
[[ 0.61095095  0.38904911]]
[[ 0.50372243  0.49627757]]
[[ 0.40399411  0.59600586]]
[[ 0.49483505  0.50516492]]
[[ 0.60554045  0.39445946]]
[[ 0.49857661  0.50142342]]
[[ 0.60982013  0.39017981]]
[[ 0.50316077  0.49683926]]
[[ 0.40367824  0.59632176]]
[[ 0.49407098  0.50592899]]
[[ 0.60381907  0.39618099]]
[[ 0.49749896  0.50250101]]
[[ 0.60769975  0.39230

[[ 0.60424346  0.39575654]]
[[ 0.49505034  0.50494963]]
[[ 0.61239815  0.38760179]]
[[ 0.50291264  0.49708739]]
[[ 0.39873078  0.60126919]]
[[ 0.49584532  0.50415462]]
[[ 0.61193961  0.38806033]]
[[ 0.50306433  0.49693561]]
[[ 0.39854127  0.60145873]]
[[ 0.49534863  0.50465131]]
[[ 0.61028814  0.38971183]]
[[ 0.50194931  0.49805066]]
[[ 0.39716586  0.60283411]]
[[ 0.49365667  0.50634331]]
[[ 0.6074397  0.3925603]]
[[ 0.49953336  0.50046664]]
[[ 0.61480433  0.3851957 ]]
[[ 0.50682586  0.49317414]]
[[ 0.4021104   0.59788954]]
[[ 0.49917009  0.50082994]]
[[ 0.61361033  0.38638967]]
[[ 0.50609243  0.49390754]]
[[ 0.40126854  0.59873152]]
[[ 0.49811745  0.50188255]]
[[ 0.61174178  0.38825825]]
[[ 0.50463766  0.49536231]]
[[ 0.39973673  0.6002633 ]]
[[ 0.49641868  0.50358135]]
[[ 0.60916263  0.39083737]]
[[ 0.50242496  0.49757501]]
[[ 0.39746612  0.60253388]]
[[ 0.49400339  0.50599658]]
[[ 0.60578865  0.39421135]]
[[ 0.49939451  0.50060546]]
[[ 0.61290967  0.38709027]]
[[ 0.50644851  0.49355

[[ 0.51407373  0.48592627]]
[[ 0.47575897  0.52424103]]
[[ 0.50549763  0.49450237]]
[[ 0.47359884  0.52640116]]
[[ 0.4972586  0.5027414]]
[[ 0.50989437  0.49010563]]
[[ 0.49228364  0.50771642]]
[[ 0.50366515  0.49633482]]
[[ 0.49062899  0.50937098]]
[[ 0.49842709  0.50157291]]
[[ 0.50641602  0.49358401]]
[[ 0.49488145  0.50511855]]
[[ 0.502119    0.49788105]]
[[ 0.4926233   0.50737673]]
[[ 0.49891731  0.50108272]]
[[ 0.50363523  0.49636474]]
[[ 0.49667874  0.50332123]]
[[ 0.50089359  0.49910641]]
[[ 0.49517125  0.50482869]]
[[ 0.498712  0.501288]]
[[ 0.50026971  0.49973023]]
[[ 0.49678457  0.50321543]]
[[ 0.49823207  0.50176799]]
[[ 0.49854347  0.5014565 ]]
[[ 0.49782073  0.50217932]]
[[ 0.49664193  0.50335807]]
[[ 0.49456891  0.50543112]]
[[ 0.49181727  0.50818282]]
Average Score  185.48
Choice 1: 0.5024261375889584, Choice 0: 0.4975738624110416
